# Database Structure Analysis and Documentation
---

**Author:** Miguel Ángel del Rincón Manrique  
**Department:** Data Science & Analytics  
**Creation Date:** June 21, 2024  
**Last Updated:** June 26, 2024  
**Version:** 0.7  

## Document Information
- **Status:** Production
- **Classification:** Internal
- **Technologies:** Python 3.X, MySQL
- **Dependencies:** mysql-connector-python, csv, logging, multiprocessing
- **Recommended IDE:** Jupyter Notebook / VS Code

## Version Control
- v0.1 (06/21/2024): Initial Release
  - Basic MySQL database connection
  - Simple table and column scanning
  - Basic CSV file generation

- v0.2 (06/21/2024): Parallel Processing
  - Added multiprocessing capabilities
  - Process pool implementation
  - Performance optimization for large databases

- v0.3 (06/24/2024): Encoding Management
  - UTF-8 and Latin1 encoding handling
  - Special characters cleaning
  - Data truncation implementation

- v0.4 (06/24/2024): HTML Processing
  - HTML entities decoding
  - Line break handling
  - Special characters management

- v0.5 (06/24/2024): Error Management
  - Logging system implementation
  - Enhanced exception handling
  - Error recovery capabilities

- v0.6 (06/25/2024): File Processing
  - File extension search functionality
  - Results filtering
  - Specific report generation

- v0.7 (06/26/2024): Current Version
  - Resume functionality for interrupted scans
  - Progress saving implementation
  - Checkpoint system

## Usage and License
This notebook is intended for internal use. All rights reserved.
Do not distribute without authorization.

---

## Overview

This Python script is designed to perform a comprehensive analysis of a MySQL database structure and content. It's particularly useful for database administrators, developers, and data analysts who need to understand, document, or track changes in large and complex database systems. The script offers a range of features to handle various data complexities and provides flexible output options.

## Key Features and Functionalities

### 1. Database Structure Analysis
- Scans all tables and columns in the specified database.
- Retrieves detailed information including table names, column names, data types, primary and foreign key relationships.

### 2. Data Sampling
- For each column, the script retrieves sample data (up to two examples per column).
- This feature helps in understanding the actual content stored in each field.

### 3. File Extension Filtering
- Allows searching for specific file types within text fields.
- Useful for identifying columns that might contain links or references to files of particular types (e.g., PDFs, images).

### 4. Advanced Data Handling

#### a. Encoding Issue Management
- Handles potential encoding issues by attempting to decode strings using both 'latin1' and 'utf-8' encodings.
- This approach helps in correctly interpreting characters from different language sets.

#### b. HTML Entity Unescaping
- Unescapes HTML entities found in the data.
- Converts HTML encoded characters (like &amp;) back to their original form for better readability.

#### c. Newline Handling
- Replaces newlines with spaces in data fields.
- Ensures that multi-line content doesn't break the structure of the output CSV file.

#### d. Value Truncation
- Truncates long values to 1000 characters.
- Prevents excessively long fields from dominating the output while still providing a substantial sample.

### 5. Multiprocessing
- Utilizes parallel processing to improve performance, especially beneficial for large databases.

### 6. Error Handling and Continuity
- Implements robust error handling to continue processing even if issues occur with specific tables or columns.

### 7. Resume Capability
- Allows for interruption and resumption of long-running scans.
- Useful for very large databases where the scan might take a considerable amount of time.

### 8. Comparison Feature
- Can compare current scan results with previous scans.
- Identifies and reports on changes in the database structure or content between scans.

### 9. Flexible Output
- Generates a comprehensive CSV file containing the database structure and sample data.
- When using file extension search, produces an additional CSV file with all matching entries.
- Outputs comparison results in a markdown-formatted table, showing added, removed, and modified rows.

## Usage in Jupyter Notebook

To use this script in a Jupyter Notebook environment, you would typically import the necessary functions and then call them with the required parameters. Here's an example of how you might use it:

```python
# Import the main function from your script
from your_script_name import scan_database

# Call the function with the database connection details
scan_database(
    host='X.X.X.X',
    user='user-bewanted',
    password='password',
    database='production_schema',
    compare_file="database_structure_production_schema_X.X.X.X_20240626_113856.csv"
)


In [ ]:
conda install mysql

Retrieving notices: ...working... done
Channels:
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/miguelang611/miniconda3/envs/faster-whisper

  added / updated specs:
    - mysql


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.7.2   |       h06a4308_0         127 KB
    libcurl-8.7.1              |       h251f7ec_0         424 KB
    libssh2-1.11.0             |       h251f7ec_0         282 KB
    lz4-c-1.9.4                |       h6a678d5_1         156 KB
    mysql-8.4.0                |       h0bac5ae_0        56.6 MB
    zstd-1.5.2                 |       ha4553b6_0         488 KB
    ------------------------------------------------------------
                                           Total:        58.0 MB

The following NEW packages will be INSTALLED:

  c-ares             pkgs/main/l

In [ ]:
conda install -c anaconda mysql-connector-python


Channels:
 - anaconda
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/miguelang611/miniconda3/envs/faster-whisper

  added / updated specs:
    - mysql-connector-python


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    mysql-common-8.3.0         |       hf1915f5_4         766 KB  conda-forge
    mysql-connector-python-8.3.0|  py312hb06c811_0         767 KB  conda-forge
    mysql-libs-8.3.0           |       hca2cd23_4         1.5 MB  conda-forge
    zstd-1.5.6                 |       ha6fb4c9_0         542 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

  mysql-common       conda-forge/linux-64::mysql-common-8.3.0-hf1915f5_4 
  mysql-connector-p~ conda-forge/linux-64::my

In [1]:
import mysql.connector
import csv
import html
import logging
import argparse
import sys
import re
from datetime import datetime
from multiprocessing import Pool, cpu_count
from functools import partial

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def connect_db(host, user, password, database):
    return mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )

def get_tables(cursor, database):
    cursor.execute(f"SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = '{database}'")
    return [table[0] for table in cursor.fetchall()]

def clean_value(value):
    if value is None:
        return ''
    try:
        value = str(value).encode('latin1').decode('utf-8')
    except:
        value = str(value)
    value = html.unescape(value)
    value = value.replace('\n', ' ').replace('\r', '')
    return value[:1000] if len(value) > 1000 else value

def is_valid_file(filename, search_extensions):
    if not search_extensions:
        return True
    valid_extensions = re.compile(r'.*\.(' + '|'.join(search_extensions) + ')$', re.IGNORECASE)
    return bool(valid_extensions.match(filename))

def process_table(table_name, host, user, password, database, search_extensions):
    try:
        conn = connect_db(host, user, password, database)
        cursor = conn.cursor()

        cursor.execute(f"""
            SELECT 
                c.COLUMN_NAME, 
                c.DATA_TYPE,
                c.IS_NULLABLE,
                c.COLUMN_KEY,
                kcu.REFERENCED_TABLE_NAME,
                kcu.REFERENCED_COLUMN_NAME,
                c.COLUMN_COMMENT
            FROM 
                INFORMATION_SCHEMA.COLUMNS c
            LEFT JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE kcu
                ON c.TABLE_SCHEMA = kcu.TABLE_SCHEMA
                AND c.TABLE_NAME = kcu.TABLE_NAME
                AND c.COLUMN_NAME = kcu.COLUMN_NAME
                AND kcu.REFERENCED_TABLE_NAME IS NOT NULL
            WHERE 
                c.TABLE_SCHEMA = '{database}' 
                AND c.TABLE_NAME = '{table_name}'
        """)
        columns = cursor.fetchall()

        results = []
        all_matches = []
        for column in columns:
            column_name, data_type, is_nullable, column_key, referenced_table, referenced_column, column_comment = column
            is_unique = 'YES' if column_key == 'UNI' else 'NO'
            is_primary_key = 'YES' if column_key == 'PRI' else 'NO'
            is_foreign_key = 'YES' if referenced_table and referenced_column else 'NO'
            referenced_table_column = f"{referenced_table}.{referenced_column}" if referenced_table and referenced_column else ''

            if search_extensions:
                like_conditions = " OR ".join([f"`{column_name}` LIKE '%.{ext}%'" for ext in search_extensions])
                query = f"SELECT `{column_name}` FROM `{table_name}` WHERE {like_conditions}"
            else:
                query = f"SELECT `{column_name}` FROM `{table_name}` LIMIT 2"

            cursor.execute(query)
            examples = cursor.fetchall()
            
            valid_examples = [clean_value(ex[0]) for ex in examples if is_valid_file(str(ex[0]), search_extensions)]
            
            if search_extensions:
                for example in valid_examples:
                    all_matches.append([table_name, column_name, example])
            
            example1 = valid_examples[0] if valid_examples else ''
            example2 = valid_examples[1] if len(valid_examples) > 1 else ''

            # Generate a description using the available information
            description = f"This column '{column_name}' is of type {data_type}. "
            description += "It can contain NULL values. " if is_nullable == 'YES' else "It cannot contain NULL values. "
            if is_unique == 'YES':
                description += "It has a unique constraint. "
            if is_primary_key == 'YES':
                description += "It is the primary key of the table. "
            if is_foreign_key == 'YES':
                description += f"It is a foreign key referencing {referenced_table_column}. "
            if column_comment:
                description += f"Comment: {column_comment}"

            results.append([
                table_name, 
                column_name, 
                data_type, 
                is_nullable,
                is_unique,
                is_primary_key, 
                is_foreign_key, 
                referenced_table_column,
                example1,
                example2,
                description
            ])

        cursor.close()
        conn.close()
        return results, all_matches
    except Exception as e:
        logging.error(f"Error processing table {table_name}: {str(e)}")
        return [], []

def save_all_matches(all_matches, output_file):
    with open(output_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, quoting=csv.QUOTE_ALL, escapechar='\\', quotechar='"')
        writer.writerow(['Table', 'Column', 'Matching Data'])
        writer.writerows(all_matches)
    logging.info(f"All matches saved to '{output_file}'")

def scan_database(host, user, password, database, search_extensions=None, output_file=None, resume_file=None, compare_file=None):
    conn = connect_db(host, user, password, database)
    cursor = conn.cursor()
    tables = get_tables(cursor, database)
    cursor.close()
    conn.close()

    if not output_file:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = f"database_structure_{database}_{host}_{timestamp}.csv"

    all_results = []
    all_matches = []

    if resume_file:
        with open(resume_file, 'r', newline='', encoding='utf-8') as file:
            reader = csv.reader(file)
            next(reader)  # Skip header
            all_results = list(reader)
        processed_tables = set(row[0] for row in all_results)
        tables = [table for table in tables if table not in processed_tables]

    num_processes = cpu_count() - 1 or 1
    with Pool(processes=num_processes) as pool:
        process_table_partial = partial(process_table, host=host, user=user, password=password, database=database, search_extensions=search_extensions)
        results = pool.map(process_table_partial, tables)

    for result, matches in results:
        all_results.extend(result)
        all_matches.extend(matches)

    with open(output_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, quoting=csv.QUOTE_ALL, escapechar='\\', quotechar='"')
        writer.writerow(['Table', 'Column', 'Data Type', 'Is Nullable', 'Is Unique', 'Is Primary Key', 'Is Foreign Key', 'Referenced Table.Column', 'Example 1', 'Example 2', 'Descripción (Claude)'])
        writer.writerows(all_results)

    logging.info(f"CSV file saved as '{output_file}'")

    if search_extensions:
        all_matches_file = f"all_matches_{database}_{host}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        save_all_matches(all_matches, all_matches_file)

    if compare_file:
        compare_results(compare_file, all_results)

def compare_results(old_file, new_results):
    with open(old_file, 'r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        old_results = list(reader)

    old_dict = {(row[0], row[1]): row for row in old_results}
    new_dict = {(row[0], row[1]): row for row in new_results}

    added = [row for key, row in new_dict.items() if key not in old_dict]
    removed = [row for key, row in old_dict.items() if key not in new_dict]
    modified = [new_dict[key] for key in old_dict.keys() & new_dict.keys() if old_dict[key] != new_dict[key]]

    print("\nAdded rows:")
    print_markdown_table(added)

    print("\nRemoved rows:")
    print_markdown_table(removed)

    print("\nModified rows:")
    print_markdown_table(modified)

def print_markdown_table(results):
    headers = ["Table", "Column", "Data Type", "Is Nullable", "Is Unique", "Is Primary Key", "Is Foreign Key", "Referenced Table.Column", "Example 1", "Example 2", "Descripción (Claude)"]
    print("| " + " | ".join(headers) + " |")
    print("| " + " | ".join(["---"] * len(headers)) + " |")
    for row in results:
        print("| " + " | ".join(str(cell) for cell in row) + " |")

In [ ]:
# Call the function with the database connection details
scan_database(
    host='X.X.X.X',
    user='user-bewanted',
    password='password',
    database='production_schema',
)